In [165]:
import numpy as np
import networkx as nx
from pyvis.network import Network
import math

import seaborn as sns
import pandas as pd
import os

os.chdir(os.path.expanduser('~/Documents/vivarium-ecoli'))

import matplotlib.pyplot as plt
import dill
import xmltodict
import os
import requests
from bs4 import BeautifulSoup
from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult, MetabolismRedux


FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

In [166]:
# Change the environment name here
env_name = 'Acetate_Transport_+_Metabolism'

In [167]:
time = '50'
date = '2023-07-21'
experiment = 'fba_new_environments'
entry = f'{experiment}_{env_name}_{time}_{date}'
folder = f'out/fba_new_env/{entry}/'

f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux']
stoichiometry = metabolism.stoichiometry

output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

sim_fluxes = pd.DataFrame(fba["estimated_fluxes"], columns = metabolism.reaction_names)
sim_fluxes = pd.DataFrame(sim_fluxes.loc[24, :].abs().sort_values(ascending=False))

metabolites_idx = {species: i for i, species in enumerate(metabolism.metabolite_names)}
index_to_met = {v: k for k, v in metabolites_idx.items()}

reaction_idx = {reaction: i for i, reaction in enumerate(metabolism.reaction_names)}
index_to_rxn = {v: k for k, v in reaction_idx.items()}

metabolism.catalyst_ids = metabolism.parameters['catalyst_ids']
catalyst_idx = {catalyst: i for i, catalyst in enumerate(metabolism.catalyst_ids)}


In [168]:
heatmap_df = pd.read_excel("out/new_env_analysis/heatmap.xlsx")

## Building the graph

In [169]:
G = nx.MultiDiGraph()

met_length = len(metabolism.metabolite_names)
reaction_length = np.shape(stoichiometry)[1]
met_not_include = ["ATP[c]", "WATER[c]", "PROTON[p]", "Pi[c]", "PROTON[c]"]
# met_not_include = []
do_not_include = []

for met in met_not_include:
    do_not_include.append(metabolites_idx[met])

def plot_metabolite(G, metabolite):
    index = metabolites_idx[metabolite]
    for k in range(reaction_length):
        if stoichiometry[index, k] > 0:
            for l in range(met_length):
                if stoichiometry[l, k] < 0:
                    if l != index and l not in do_not_include:
                        G.add_edge(index_to_met[l], metabolite) # Add weight here equal to reaction flux
        elif stoichiometry[index, k] < 0:
            for l in range(met_length):
                if stoichiometry[l, k] > 0:
                    if l != index and l not in do_not_include:
                        G.add_edge(index_to_met[l], metabolite) # Add weight here equal to reaction flux


def plot_reaction(G, reaction, flux, color):
    index = reaction_idx[reaction]
    reactants = []
    products = []
    clean_name = clean_reaction_name(reaction)
    for k in range(met_length):
        if stoichiometry[k, index] > 0 and index_to_met[k] not in met_not_include:
            products.append(index_to_met[k])
        elif stoichiometry[k, index] < 0 and index_to_met[k] not in met_not_include:
            reactants.append(index_to_met[k])

    for reactant in reactants:
        G.add_edge(reactant, clean_name, color=color, title=flux)

    for product in products:
        G.add_edge(product, clean_name, color=color, title=flux)

In [170]:
# Not currently used in this analysis but can be useful later

def clean_reaction_name(reaction):
    if '/' in reaction:
        index = reaction.find('RXN')
        if (reaction[index+3] == '-' and reaction[index+4].isdigit()) or (reaction[index+4] == '-' and reaction[index+5].isdigit()):
            index = index + 4
            if reaction[index] == '-':
                index += 1
            while reaction[index].isdigit():
                index += 1
            reaction = reaction[:index]
        else:
            reaction = reaction[:index+3]
    elif '__' in reaction:
        reaction = reaction[:reaction.find('__')]
    elif '(reverse)' in reaction:
        reaction = reaction[:reaction.find('(reverse)') - 1]
    elif '[' in reaction:
        reaction = reaction[:reaction.find('[') - 1]
    elif ' ' in reaction:
        reaction = reaction[:reaction.find('[')]
    return reaction


In [171]:
k = 20
heatmap_df.sort_values(by=[env_name], ascending=False, inplace=True)
top_increase = [heatmap_df.iloc[i, 0] for i in range(k)]
bottom_increase = [heatmap_df.iloc[i, 0] for i in range(-1, -1*k - 1, -1)]

if "Non-zero Values for each environment" in top_increase:
    top_increase.remove("Non-zero Values for each environment")
elif "Non-zero Values for each environment" in bottom_increase:
    bottom_increase.remove("Non-zero Values for each environment")

for reaction in top_increase:
    row_num = heatmap_df[heatmap_df['Reactions'] == reaction].index[0]
    plot_reaction(G, reaction, heatmap_df[env_name].loc[row_num], 'green')

for reaction in bottom_increase:
    row_num = heatmap_df[heatmap_df['Reactions'] == reaction].index[0]
    plot_reaction(G, reaction, heatmap_df[env_name].loc[row_num], 'red')

# nx.write_graphml(G, f'out/new_env_analysis/{env_name}_top10_graph.graphml')

nt = Network('1000px', '2000px', notebook=True)
nt.repulsion()
nt.from_nx(G)
# for e in nt.edges:
#     e['value'] = e['width']

# nt.toggle_physics(True)
nt.show(f'out/new_env_analysis/flux_visualization/{env_name}_top20_network.html')


# plt.figure(figsize=(25, 25))
# nx.draw_spring(G, with_labels=True, font_size=10, node_size=1000, node_color='lightblue', edge_color='black', width=1.5)
# plt.savefig(f'out/new_env_analysis/{env_name}_top10_graph.png', dpi=300)
# plt.show()

out/new_env_analysis/flux_visualization/Acetate_Transport_+_Metabolism_top20_network.html
